### Import Required Libraries and Set Up Environment Variables

In [ ]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os

In [ ]:
# Set environment variables from the .env in the local environment
load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')

### CME Data

In [ ]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI"

# Set the specifier for CMEs:
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for CME
cme_url = f"{base_url}/{CME}?api_key={NASA_API_KEY}&startDate={startDate}&endDate={endDate}"

def convertToDF(dfJSON):
    return(pd.json_normalize(dfJSON))

In [ ]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
# NOTE-cme_response not needed, conversion done in one line and set to cme_json
cme_json = requests.get(cme_url).json()

In [ ]:
# All elements - Use json.dumps with argument indent=4 to format data
print(json.dumps(cme_json, indent=4))

In [ ]:
def convertToDF(dfJSON):
    return(pd.json_normalize(dfJSON))
# Convert cme_json to a Pandas DataFrame 
# Keep only the columns: activityID, startTime, linkedEvents
cme_df = convertToDF(cme_json)
cme_df = cme_df[['activityID', 'startTime','linkedEvents']]
cme_df


In [ ]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
cme_df.dropna(subset=['linkedEvents'], inplace=True)
cme_df

In [ ]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Create a new DataFrame from the expanded rows
# Initialize an empty list to store the expanded rows
expanded_rows = []

# Iterate over each index in the DataFrame
for i in cme_df.index:
    activityID       = cme_df.loc[i, 'activityID']         # Get the corresponding value from 'activityID'
    startTime        = cme_df.loc[i, 'startTime']     # Get the corresponding value from 'startTime'    
    linkedEvents_col = cme_df.loc[i, 'linkedEvents']  # Get the list of dictionaries in 'linkedEvents'
    
    # Iterate over each dictionary in the list
    for item in linkedEvents_col:
        # Create a new row with the dictionary and corresponding 'activityID' and 'startTime' value
        expanded_rows.append({'activityID': activityID, 'startTime': startTime, 'linkedEvents': item})

# Create a new DataFrame from the expanded rows
cme_df_ex = pd.DataFrame(expanded_rows)

# Use the head function to show the dataframe
cme_df_ex.head()

In [ ]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# Be sure to use a try and except block to handle errors
# Log the error or print it for debugging
def extract_activityID_from_dict(input_dict):
    try:
        activityID = input_dict.get("activityID", None)
        return activityID
    except (ValueError, TypeError) as e:
        # Log the error or print it for debugging
        print(f"Error processing input dictionary: {input_dict}. Error: {e}")
        return None

# Verify that it works as expected using one row from linkedEvents as an example    
extract_activityID_from_dict(cme_df_ex.loc[0,'linkedEvents'])

In [ ]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
cme_df_ex['GST_ActivityID'] = cme_df_ex['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))

In [ ]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:
cme_df_ex.dropna(subset=['GST_ActivityID'], inplace=True)
cme_df_ex

In [ ]:
# print out the datatype of each column in this DataFrame:
#print(df_for_loop.dtypes)
cme_df_ex.info()

In [ ]:
# Convert the 'GST_ActivityID' column to string format 
cme_df_ex['GST_ActivityID'] = pd.Series(cme_df_ex["GST_ActivityID"], dtype="string")
# Convert startTime to datetime format  
cme_df_ex['startTime'] = pd.to_datetime(cme_df_ex['startTime'], format="%Y-%m-%dT%H:%MZ", errors='coerce')

In [ ]:
# Rename startTime to startTime_CME and activityID to cmeID
cme_df_ex.rename(columns={"activityID": "cmeID", "startTime": "startTime_CME"}, inplace=True)    
# Drop linkedEvents
cme_df_ex=cme_df_ex.drop(columns=["linkedEvents"])
cme_df_ex

In [ ]:
# print out the datatype of each column in this DataFrame:
cme_df_ex.info()

In [ ]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.  
cme_df_ex = cme_df_ex[cme_df_ex['GST_ActivityID'].str.contains('GST')]
cme_df_ex

### GST Data

In [ ]:
# Build URL for GST
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI"

# Set the specifier for CMEs:
GST = "GST"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for GST
gst_url = f"{base_url}/{GST}?api_key={NASA_API_KEY}&startDate={startDate}&endDate={endDate}"
print(gst_url)

In [ ]:
# Make a "GET" request for the CME URL and store it in a variable named gst_response
# NOTE-gst_response not needed, conversion done in one line and set to gst_json
gst_json = requests.get(gst_url).json()

In [ ]:
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(gst_json, indent=4))

In [ ]:
def convertToDF(dfJSON):
    return(pd.json_normalize(dfJSON))
# Convert cme_json to a Pandas DataFrame 
# Keep only the columns: gstID, startTime, linkedEvents
gst_df = convertToDF(gst_json)
gst_df = gst_df[['gstID', 'startTime','linkedEvents']]
gst_df

In [ ]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
gst_df.dropna(subset=['linkedEvents'], inplace=True)
gst_df


In [ ]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
gst_df = gst_df.explode('linkedEvents')
# Reset the index
gst_df = gst_df.reset_index(drop=True).dropna()
# Use the head function to show the dataframe
gst_df.head()

In [ ]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
def extract_activityID_from_dict(input_dict):
    try:
        activityID = input_dict.get("activityID", None)
        return activityID
    except (ValueError, TypeError) as e:
        # Log the error or print it for debugging
        print(f"Error processing input dictionary: {input_dict}. Error: {e}")
        return None
        
# and create a new column called 'CME_ActivityID':
gst_df['CME_ActivityID'] = gst_df['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))

In [ ]:
# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:
gst_df.dropna(subset=['CME_ActivityID'], inplace=True)
gst_df

In [ ]:
# Convert the 'CME_ActivityID' column to string format
gst_df['CME_ActivityID'] = pd.Series(gst_df["CME_ActivityID"], dtype="string")
# Convert the 'gstID' column to string format 
gst_df['gstID'] = pd.Series(gst_df["gstID"], dtype="string")
# Convert startTime to datetime format  
gst_df['startTime'] = pd.to_datetime(gst_df['startTime'], format="%Y-%m-%dT%H:%MZ", errors='coerce')

In [ ]:
# Rename startTime to startTime_GST
gst_df.rename(columns={"startTime":"startTime_GST"}, inplace=True)

# Drop linkedEvents
gst_df=gst_df.drop(columns=["linkedEvents"])

# Verify that all steps were executed correctly
gst_df

In [ ]:
gst_df.info()
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  
gst_df = gst_df[gst_df['CME_ActivityID'].str.contains('CME')]
gst_df

### Merge both datatsets

In [ ]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.
merged_df = pd.merge(cme_df_ex,gst_df,left_on=['GST_ActivityID','cmeID'],right_on=['gstID','CME_ActivityID'])
display(merged_df)

In [ ]:
merged_df.info()
gst_df.info()
cme_df_ex.info()

In [ ]:
# Verify that the new DataFrame has the same number of rows as cme and gst
merged_row_count = len(merged_df)
print(merged_row_count)
cme_row_count = len(cme_df_ex)
print(cme_row_count)
gst_row_count = len(gst_df)
print(gst_row_count)

### Computing the time it takes for a CME to cause a GST

In [ ]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.
merged_df['timediff'] = merged_df.apply(lambda x: x["startTime_CME"] - x["startTime_GST"], axis=1)
display(merged_df)

In [ ]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 

description = merged_df.describe()
mean_of_timediff = description['timediff']['mean']
print(mean_of_timediff)
median_of_timediff = description.loc['50%','timediff']
print(median_of_timediff)


### Exporting data in csv format

In [ ]:
# Export data to CSV without the index
# Save the DataFrame as a CSV
# Note: To avoid any issues later, use encoding="utf-8"
merged_df.to_csv("retrieve_data_solution_output.csv", encoding="utf-8", index=False)